In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import os
from typing import Dict, List
from tkinter import scrolledtext

class FileAnalyzerMerger(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.title("File Analyzer & Merger")
        self.configure(bg='#1e1e1e')
        
        # Store loaded dataframes and common columns
        self.dataframes: Dict[str, pd.DataFrame] = {}
        self.merged_file_path = None
        self.common_columns = set()
        
        # Configure style
        style = ttk.Style()
        style.theme_use('default')
        style.configure('Custom.TButton',
                       background='#333333',
                       foreground='#ffffff',
                       padding=5)
        style.configure('Custom.TFrame',
                       background='#1e1e1e')
        style.configure('Custom.TLabel',
                       background='#1e1e1e',
                       foreground='#ffffff')
        style.configure('Custom.TListbox',
                       background='#2d2d2d',
                       foreground='#ffffff')
        
        self._create_widgets()
        self._setup_layout()

    def _create_widgets(self):
        # Main frame
        self.main_frame = ttk.Frame(self, style='Custom.TFrame')
        
        # Left panel for file list
        self.left_panel = ttk.Frame(self.main_frame, style='Custom.TFrame')
        self.load_button = ttk.Button(
            self.left_panel,
            text="Load Files",
            command=self.load_files,
            style='Custom.TButton'
        )
        
        # Listbox for loaded files
        self.file_listbox = tk.Listbox(
            self.left_panel,
            bg='#2d2d2d',
            fg='#ffffff',
            selectmode=tk.MULTIPLE,
            height=10,
            width=30
        )
        self.clear_button = ttk.Button(
            self.left_panel,
            text="Clear Selected",
            command=self.clear_selected_files,
            style='Custom.TButton'
        )
        
        # Common columns frame
        self.columns_frame = ttk.Frame(self.left_panel, style='Custom.TFrame')
        self.columns_label = ttk.Label(
            self.columns_frame,
            text="Common Columns:",
            style='Custom.TLabel'
        )
        self.columns_listbox = tk.Listbox(
            self.columns_frame,
            bg='#2d2d2d',
            fg='#ffffff',
            height=5,
            width=30
        )
        self.columns_listbox.bind('<<ListboxSelect>>', self.on_column_select)
        
        # Right panel for file information
        self.right_panel = ttk.Frame(self.main_frame, style='Custom.TFrame')
        self.info_text = scrolledtext.ScrolledText(
            self.right_panel,
            bg='#2d2d2d',
            fg='#ffffff',
            height=20,
            width=50
        )
        
        # Merge section
        self.merge_frame = ttk.Frame(self.main_frame, style='Custom.TFrame')
        
        # Column selection
        self.column_frame = ttk.Frame(self.merge_frame, style='Custom.TFrame')
        self.merge_label = ttk.Label(
            self.column_frame,
            text="Merge on column:",
            style='Custom.TLabel'
        )
        self.merge_column = ttk.Entry(self.column_frame)
        
        # Merge buttons frame
        self.merge_buttons_frame = ttk.Frame(self.merge_frame, style='Custom.TFrame')
        self.merge_button = ttk.Button(
            self.merge_buttons_frame,
            text="Preview Merge",
            command=self.preview_merge,
            style='Custom.TButton'
        )
        self.save_merge_button = ttk.Button(
            self.merge_buttons_frame,
            text="Save Merged File",
            command=self.save_merged_file,
            style='Custom.TButton'
        )
        
        # Location frame
        self.location_frame = ttk.Frame(self.main_frame, style='Custom.TFrame')
        self.location_label = ttk.Label(
            self.location_frame,
            text="Merged File Location:",
            style='Custom.TLabel'
        )
        self.location_text = tk.Text(
            self.location_frame,
            bg='#2d2d2d',
            fg='#ffffff',
            height=1,
            width=50
        )
        
        # Store the merged dataframe
        self.merged_df = None

    def _setup_layout(self):
        # Configure main window
        self.geometry("1000x800")
        
        # Pack main frame
        self.main_frame.pack(expand=True, fill='both', padx=10, pady=10)
        
        # Pack left panel
        self.left_panel.pack(side='left', fill='y', padx=(0, 10))
        self.load_button.pack(pady=(0, 5))
        self.file_listbox.pack(fill='both', expand=True)
        self.clear_button.pack(pady=(5, 0))
        
        # Pack columns frame
        self.columns_frame.pack(fill='x', pady=10)
        self.columns_label.pack()
        self.columns_listbox.pack(fill='both', expand=True)
        
        # Pack right panel
        self.right_panel.pack(side='left', fill='both', expand=True)
        self.info_text.pack(fill='both', expand=True)
        
        # Pack merge section
        self.merge_frame.pack(fill='x', pady=10)
        
        # Pack column selection
        self.column_frame.pack(fill='x', pady=5)
        self.merge_label.pack(side='left', padx=5)
        self.merge_column.pack(side='left', padx=5, fill='x', expand=True)
        
        # Pack merge buttons
        self.merge_buttons_frame.pack(fill='x', pady=5)
        self.merge_button.pack(side='left', padx=5)
        self.save_merge_button.pack(side='left', padx=5)
        self.save_merge_button.config(state='disabled')  # Initially disabled
        
        # Pack location frame
        self.location_frame.pack(fill='x', pady=5)
        self.location_label.pack(side='left', padx=5)
        self.location_text.pack(side='left', padx=5, fill='x', expand=True)
        self.location_text.config(state='disabled')

    def update_common_columns(self):
        """Update the list of common columns across all loaded files"""
        if not self.dataframes:
            self.common_columns = set()
        else:
            # Get columns from all dataframes
            all_columns = [set(df.columns) for df in self.dataframes.values()]
            # Find intersection of all column sets
            self.common_columns = set.intersection(*all_columns)
        
        # Update the columns listbox
        self.columns_listbox.delete(0, tk.END)
        for column in sorted(self.common_columns):
            self.columns_listbox.insert(tk.END, column)
            
        # Display common columns info
        if self.common_columns:
            self.info_text.insert(tk.END, "\n=== Common Columns ===\n")
            self.info_text.insert(tk.END, ", ".join(sorted(self.common_columns)) + "\n")

    def on_column_select(self, event):
        """Handle column selection from the listbox"""
        selection = self.columns_listbox.curselection()
        if selection:
            selected_column = self.columns_listbox.get(selection[0])
            self.merge_column.delete(0, tk.END)
            self.merge_column.insert(0, selected_column)

    def clear_selected_files(self):
        selected_indices = self.file_listbox.curselection()
        selected_files = [self.file_listbox.get(i) for i in selected_indices]
        
        # Remove from dataframes dictionary
        for file in selected_files:
            if file in self.dataframes:
                del self.dataframes[file]
        
        # Remove from listbox
        for index in selected_indices[::-1]:
            self.file_listbox.delete(index)
        
        # Update info display and common columns
        self.update_info_display()
        self.update_common_columns()
        
        # Reset merge state
        self.merged_df = None
        self.save_merge_button.config(state='disabled')
        self.location_text.config(state='normal')
        self.location_text.delete(1.0, tk.END)
        self.location_text.config(state='disabled')

    def update_info_display(self):
        self.info_text.delete(1.0, tk.END)
        
        for filename, df in self.dataframes.items():
            # Calculate file info
            file_path = os.path.join(os.getcwd(), filename)
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / 1024  # KB
            else:
                file_size = 0
                
            memory_usage = df.memory_usage(deep=True).sum() / 1024  # KB
            
            # Display file information
            info = (f"File: {filename}\n"
                   f"Size: {file_size:.2f} KB\n"
                   f"Rows: {len(df)}\n"
                   f"Columns: {len(df.columns)}\n"
                   f"Memory Usage: {memory_usage:.2f} KB\n"
                   f"Column Names: {', '.join(df.columns)}\n\n")
            
            self.info_text.insert(tk.END, info)

    def load_files(self):
        files = filedialog.askopenfilenames(
            filetypes=[
                ("CSV files", "*.csv"),
                ("Excel files", "*.xlsx;*.xls")
            ]
        )
        
        for file in files:
            try:
                # Load file based on extension
                if file.endswith(('.xlsx', '.xls')):
                    df = pd.read_excel(file)
                else:
                    df = pd.read_csv(file)
                
                # Store dataframe
                filename = os.path.basename(file)
                self.dataframes[filename] = df
                
                # Add to listbox if not already present
                if filename not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, filename)
                
            except Exception as e:
                messagebox.showerror("Error", f"Error loading {file}: {str(e)}")
        
        # Update info display and common columns
        self.update_info_display()
        self.update_common_columns()

    def preview_merge(self):
        if len(self.dataframes) < 2:
            messagebox.showwarning("Warning", "Please load at least 2 files to merge.")
            return
        
        merge_col = self.merge_column.get().strip()
        if not merge_col:
            messagebox.showwarning("Warning", "Please specify a merge column.")
            return
        
        try:
            # Check if merge column exists in all dataframes
            for name, df in self.dataframes.items():
                if merge_col not in df.columns:
                    messagebox.showerror("Error", f"Column '{merge_col}' not found in {name}")
                    return
            
            # Merge all dataframes
            result = None
            for df in self.dataframes.values():
                if result is None:
                    result = df
                else:
                    result = result.merge(df, on=merge_col, how='outer')
            
            # Store merged dataframe
            self.merged_df = result
            
            # Enable save button
            self.save_merge_button.config(state='normal')
            
            # Display preview info
            preview_info = ("\n=== Merge Preview ===\n"
                          f"Total Rows: {len(result)}\n"
                          f"Total Columns: {len(result.columns)}\n"
                          f"Memory Usage: {result.memory_usage(deep=True).sum() / 1024:.2f} KB\n"
                          f"Columns: {', '.join(result.columns)}\n")
            
            self.info_text.insert(tk.END, preview_info)
            
        except Exception as e:
            messagebox.showerror("Error", f"Error during merge preview: {str(e)}")

    def save_merged_file(self):
        if self.merged_df is None:
            messagebox.showwarning("Warning", "Please preview the merge first.")
            return
        
        try:
            # Save merged file
            save_path = filedialog.asksaveasfilename(
                defaultextension=".csv",
                filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")]
            )
            
            if save_path:
                if save_path.endswith('.csv'):
                    self.merged_df.to_csv(save_path, index=False)
                else:
                    self.merged_df.to_excel(save_path, index=False)
                
                # Update location text
                self.location_text.config(state='normal')
                self.location_text.delete(1.0, tk.END)
                self.location_text.insert(1.0, save_path)
                self.location_text.config(state='disabled')
                
                # Store the path
                self.merged_file_path = save_path
                
                messagebox.showinfo("Success", "Files merged and saved successfully!")
                
                # Display final file info
                file_size = os.path.getsize(save_path) / 1024  # KB
                
                final_info = ("\n=== Final Merged File Info ===\n"
                             f"Location: {save_path}\n"
                             f"Size: {file_size:.2f} KB\n"
                             f"Rows: {len(self.merged_df)}\n"
                             f"Columns: {len(self.merged_df.columns)}\n")
                
                self.info_text.insert(tk.END, final_info)
                
        except Exception as e:
            messagebox.showerror("Error", f"Error saving merged file: {str(e)}")

if __name__ == "__main__":
    app = FileAnalyzerMerger()
    app.mainloop()

In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import os
from typing import Dict, List
from tkinter import scrolledtext

class FileAnalyzerMerger(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.title("File Analyzer & Merger")
        self.configure(bg='#1e1e1e')
        
        # Store loaded dataframes and common columns
        self.dataframes: Dict[str, pd.DataFrame] = {}
        self.merged_file_path = None
        self.common_columns = set()
        self.merged_df = None
        
        # Configure style
        style = ttk.Style()
        style.theme_use('default')
        style.configure('Custom.TButton',
                       background='#333333',
                       foreground='#ffffff',
                       padding=5)
        style.configure('Custom.TFrame',
                       background='#1e1e1e')
        style.configure('Custom.TLabel',
                       background='#1e1e1e',
                       foreground='#ffffff')
        style.configure('Custom.TListbox',
                       background='#2d2d2d',
                       foreground='#ffffff')
        
        self._create_widgets()
        self._setup_layout()

    def _create_widgets(self):
        # Main frame
        self.main_frame = ttk.Frame(self, style='Custom.TFrame')
        
        # Left panel for file list
        self.left_panel = ttk.Frame(self.main_frame, style='Custom.TFrame')
        self.load_button = ttk.Button(
            self.left_panel,
            text="Load Files",
            command=self.load_files,
            style='Custom.TButton'
        )
        
        # Listbox for loaded files
        self.file_listbox = tk.Listbox(
            self.left_panel,
            bg='#2d2d2d',
            fg='#ffffff',
            selectmode=tk.MULTIPLE,
            height=10,
            width=30
        )
        self.clear_button = ttk.Button(
            self.left_panel,
            text="Clear Selected",
            command=self.clear_selected_files,
            style='Custom.TButton'
        )
        
        # Common columns frame
        self.columns_frame = ttk.Frame(self.left_panel, style='Custom.TFrame')
        self.columns_label = ttk.Label(
            self.columns_frame,
            text="Common Columns:",
            style='Custom.TLabel'
        )
        self.columns_listbox = tk.Listbox(
            self.columns_frame,
            bg='#2d2d2d',
            fg='#ffffff',
            height=5,
            width=30
        )
        self.columns_listbox.bind('<<ListboxSelect>>', self.on_column_select)
        
        # Right panel for file information
        self.right_panel = ttk.Frame(self.main_frame, style='Custom.TFrame')
        self.info_text = scrolledtext.ScrolledText(
            self.right_panel,
            bg='#2d2d2d',
            fg='#ffffff',
            height=20,
            width=50
        )
        
        # Progress bar
        self.progress_var = tk.DoubleVar()
        self.progress_bar = ttk.Progressbar(
            self.right_panel,
            variable=self.progress_var,
            maximum=100,
            mode='determinate'
        )
        
        # Merge section
        self.merge_frame = ttk.Frame(self.main_frame, style='Custom.TFrame')
        
        # Column selection
        self.column_frame = ttk.Frame(self.merge_frame, style='Custom.TFrame')
        self.merge_label = ttk.Label(
            self.column_frame,
            text="Merge on column:",
            style='Custom.TLabel'
        )
        self.merge_column = ttk.Entry(self.column_frame)
        
        # Merge buttons frame
        self.merge_buttons_frame = ttk.Frame(self.merge_frame, style='Custom.TFrame')
        self.merge_button = ttk.Button(
            self.merge_buttons_frame,
            text="Preview Merge",
            command=self.preview_merge,
            style='Custom.TButton'
        )
        self.save_merge_button = ttk.Button(
            self.merge_buttons_frame,
            text="Save Merged File",
            command=self.save_merged_file,
            style='Custom.TButton'
        )
        
        # Location frame
        self.location_frame = ttk.Frame(self.main_frame, style='Custom.TFrame')
        self.location_label = ttk.Label(
            self.location_frame,
            text="Merged File Location:",
            style='Custom.TLabel'
        )
        self.location_text = tk.Text(
            self.location_frame,
            bg='#2d2d2d',
            fg='#ffffff',
            height=1,
            width=50
        )

    def _setup_layout(self):
        # Configure main window
        self.geometry("1000x800")
        
        # Pack main frame
        self.main_frame.pack(expand=True, fill='both', padx=10, pady=10)
        
        # Pack left panel
        self.left_panel.pack(side='left', fill='y', padx=(0, 10))
        self.load_button.pack(pady=(0, 5))
        self.file_listbox.pack(fill='both', expand=True)
        self.clear_button.pack(pady=(5, 0))
        
        # Pack columns frame
        self.columns_frame.pack(fill='x', pady=10)
        self.columns_label.pack()
        self.columns_listbox.pack(fill='both', expand=True)
        
        # Pack right panel
        self.right_panel.pack(side='left', fill='both', expand=True)
        self.info_text.pack(fill='both', expand=True)
        self.progress_bar.pack(fill='x', pady=5)
        
        # Pack merge section
        self.merge_frame.pack(fill='x', pady=10)
        
        # Pack column selection
        self.column_frame.pack(fill='x', pady=5)
        self.merge_label.pack(side='left', padx=5)
        self.merge_column.pack(side='left', padx=5, fill='x', expand=True)
        
        # Pack merge buttons
        self.merge_buttons_frame.pack(fill='x', pady=5)
        self.merge_button.pack(side='left', padx=5)
        self.save_merge_button.pack(side='left', padx=5)
        self.save_merge_button.config(state='disabled')
        
        # Pack location frame
        self.location_frame.pack(fill='x', pady=5)
        self.location_label.pack(side='left', padx=5)
        self.location_text.pack(side='left', padx=5, fill='x', expand=True)
        self.location_text.config(state='disabled')

    def optimize_dataframe(self, df):
        """Optimize dataframe memory usage by adjusting data types"""
        for col in df.columns:
            col_type = df[col].dtype
            
            if col_type == 'object':
                # Try to convert object to category if it has few unique values
                num_unique = df[col].nunique()
                if num_unique / len(df) < 0.5:  # If less than 50% unique values
                    df[col] = df[col].astype('category')
                    
            elif col_type.name.startswith('int'):
                # Downcast integers
                df[col] = pd.to_numeric(df[col], downcast='integer')
                
            elif col_type.name.startswith('float'):
                # Downcast floats
                df[col] = pd.to_numeric(df[col], downcast='float')
        
        return df

    def chunk_merge(self, dfs, merge_col, chunk_size=100000):
        """Merge dataframes in chunks to avoid memory issues"""
        # Sort all dataframes by merge column to optimize merge
        sorted_dfs = [df.sort_values(merge_col) for df in dfs]
        
        # Use the first dataframe as base
        result = sorted_dfs[0]
        
        # Calculate total operations for progress bar
        total_ops = len(sorted_dfs) - 1
        current_op = 0
        
        # Merge with remaining dataframes
        for df in sorted_dfs[1:]:
            # Process in chunks
            chunks = []
            total_chunks = (len(result) + chunk_size - 1) // chunk_size
            
            for chunk_idx, chunk_start in enumerate(range(0, len(result), chunk_size)):
                chunk_end = chunk_start + chunk_size
                chunk = result[chunk_start:chunk_end]
                
                # Merge chunk with the entire second dataframe
                merged_chunk = chunk.merge(df, on=merge_col, how='outer')
                chunks.append(merged_chunk)
                
                # Update progress
                progress = (current_op * 100 + (chunk_idx / total_chunks) * 100) / total_ops
                self.progress_var.set(progress)
                self.update_idletasks()
            
            # Concatenate chunks
            result = pd.concat(chunks, ignore_index=True)
            
            # Optimize memory usage after each merge
            result = self.optimize_dataframe(result)
            
            current_op += 1
        
        self.progress_var.set(100)
        return result

    def update_common_columns(self):
        """Update the list of common columns across all loaded files"""
        if not self.dataframes:
            self.common_columns = set()
        else:
            # Get columns from all dataframes
            all_columns = [set(df.columns) for df in self.dataframes.values()]
            # Find intersection of all column sets
            self.common_columns = set.intersection(*all_columns)
        
        # Update the columns listbox
        self.columns_listbox.delete(0, tk.END)
        for column in sorted(self.common_columns):
            self.columns_listbox.insert(tk.END, column)
        
        # Display common columns info
        if self.common_columns:
            self.info_text.insert(tk.END, "\n=== Common Columns ===\n")
            self.info_text.insert(tk.END, ", ".join(sorted(self.common_columns)) + "\n")

    def on_column_select(self, event):
        """Handle column selection from the listbox"""
        selection = self.columns_listbox.curselection()
        if selection:
            selected_column = self.columns_listbox.get(selection[0])
            self.merge_column.delete(0, tk.END)
            self.merge_column.insert(0, selected_column)

    def load_files(self):
        files = filedialog.askopenfilenames(
            filetypes=[
                ("CSV files", "*.csv"),
                ("Excel files", "*.xlsx;*.xls")
            ]
        )
        
        for file in files:
            try:
                # Load file based on extension
                if file.endswith(('.xlsx', '.xls')):
                    df = pd.read_excel(file)
                else:
                    df = pd.read_csv(file)
                
                # Optimize dataframe immediately after loading
                df = self.optimize_dataframe(df)
                
                # Store dataframe
                filename = os.path.basename(file)
                self.dataframes[filename] = df
                
                # Add to listbox if not already present
                if filename not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, filename)
                
            except Exception as e:
                messagebox.showerror("Error", f"Error loading {file}: {str(e)}")
        
        # Update info display and common columns
        self.update_info_display()
        self.update_common_columns()

    def clear_selected_files(self):
        selected_indices = self.file_listbox.curselection()
        selected_files = [self.file_listbox.get(i) for i in selected_indices]
        
        # Remove from dataframes dictionary
        for file in selected_files:
            if file in self.dataframes:
                del self.dataframes[file]
        
        # Remove from listbox
        for index in selected_indices[::-1]:
            self.file_listbox.delete(index)
        
        # Update info display and common columns
        self.update_info_display()
        self.update_common_columns()
        
        # Reset merge state
        self.merged_df = None
        self.save_merge_button.config(state='disabled')
        self.location_text.config(state='normal')
        self.location_text.delete(1.0, tk.END)
        self.location_text.config(state='disabled')
        self.progress_var.set(0)

    def update_info_display(self):
        self.info_text.delete(1.0, tk.END)
        
        for filename, df in self.dataframes.items():
            # Calculate file info
            file_path = os.path.join(os.getcwd(), filename)
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
            else:
                file_size = 0
            
            memory_usage = df.memory_usage(deep=True).sum() / (1024 * 1024)  # MB
            
            # Display file information
            info = (f"File: {filename}\n"
                   f"Size: {file_size:.2f} MB\n"
                   f"Rows: {len(df):,}\n"
                   f"Columns: {len(df.columns)}\n"
                   f"Memory Usage: {memory_usage:.2f} MB\n"
                   f"Column Names: {', '.join(df.columns)}\n\n")
            
            self.info_text.insert(tk.END, info)

    def preview_merge(self):
        if len(self.dataframes) < 2:
            messagebox.showwarning("Warning", "Please load at least 2 files to merge.")
            return
        
        merge_col = self.merge_column.get().strip()
        if not merge_col:
            messagebox.showwarning("Warning", "Please specify a merge column.")
            return
        
        try:
            # Reset progress bar
            self.progress_var.set(0)
            
            # Check if merge column exists in all dataframes
            for name, df in self.dataframes.items():
                if merge_col not in df.columns:
                    messagebox.showerror("Error", f"Column '{merge_col}' not found in {name}")
                    return
            
            # Optimize all dataframes before merging
            optimized_dfs = []
            for df in self.dataframes.values():
                opt_df = self.optimize_dataframe(df.copy())
                optimized_dfs.append(opt_df)
            
            # Show progress message
            self.info_text.insert(tk.END, "\nStarting merge process... This may take a while for large files.\n")
            self.update_idletasks()
            
            # Perform chunked merge
            result = self.chunk_merge(optimized_dfs, merge_col)
            
            # Store merged dataframe
            self.merged_df = result
            
            # Enable save button
            self.save_merge_button.config(state='normal')
            
            # Display preview info
            total_memory = result.memory_usage(deep=True).sum() / (1024 * 1024)  # MB
            preview_info = ("\n=== Merge Preview ===\n"
                          f"Total Rows: {len(result):,}\n"
                          f"Total Columns: {len(result.columns)}\n"
                          f"Memory Usage: {total_memory:.2f} MB\n"
                          f"Columns: {', '.join(result.columns)}\n")
            
            self.info_text.insert(tk.END, preview_info)
            
        except Exception as e:
            messagebox.showerror("Error", f"Error during merge preview: {str(e)}")
            # Print detailed error for debugging
            import traceback
            self.info_text.insert(tk.END, f"\nError details:\n{traceback.format_exc()}\n")

    def save_merged_file(self):
        if self.merged_df is None:
            messagebox.showwarning("Warning", "Please preview the merge first.")
            return
        
        try:
            # Save merged file
            save_path = filedialog.asksaveasfilename(
                defaultextension=".csv",
                filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")]
            )
            
            if save_path:
                # Show saving message
                self.info_text.insert(tk.END, "\nSaving merged file... Please wait.\n")
                self.update_idletasks()
                
                # Save based on file extension
                if save_path.endswith('.csv'):
                    # Save CSV in chunks to handle large files
                    self.merged_df.to_csv(save_path, index=False, chunksize=100000)
                else:
                    # Save Excel file
                    with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
                        self.merged_df.to_excel(writer, index=False)
                
                # Update location text
                self.location_text.config(state='normal')
                self.location_text.delete(1.0, tk.END)
                self.location_text.insert(1.0, save_path)
                self.location_text.config(state='disabled')
                
                # Store the path
                self.merged_file_path = save_path
                
                messagebox.showinfo("Success", "Files merged and saved successfully!")
                
                # Display final file info
                file_size = os.path.getsize(save_path) / (1024 * 1024)  # MB
                
                final_info = ("\n=== Final Merged File Info ===\n"
                             f"Location: {save_path}\n"
                             f"Size: {file_size:.2f} MB\n"
                             f"Rows: {len(self.merged_df):,}\n"
                             f"Columns: {len(self.merged_df.columns)}\n")
                
                self.info_text.insert(tk.END, final_info)
                
        except Exception as e:
            messagebox.showerror("Error", f"Error saving merged file: {str(e)}")
            import traceback
            self.info_text.insert(tk.END, f"\nError details:\n{traceback.format_exc()}\n")

if __name__ == "__main__":
    app = FileAnalyzerMerger()
    app.mainloop()

: 